In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import requests
import gzip
import pandas as pd
import networkx as nx
import numpy as np
import cPickle
import time
import os
import string
from itertools import islice

import ndex.client as nc

sys.path.insert(0, '/cellar/users/mikeyu/DeepTranslate/ddot')
import ddot

# Set CLIXO parameters

In [ ]:
alpha = 0.05
beta = 0.5

threads = 40

# Set NDEx account parameters

In [ ]:
ndex_server = 'http://test.ndexbio.org'
ndex_user = 'scratch'
ndex_pass = 'scratch'

# Load pre-processed Gene Ontology

In [ ]:
go_human = Ontology.from_ndex(ddot.config.GO_HUMAN_URL)
print go_human

# Load gene-gene similarity network

In [ ]:
similarity_uuid = 'd2dfa5cc-56de-11e7-a2e2-0660b7976219'
sim, sim_names = ndex_to_sim_matrix(
    similarity_uuid=similarity_uuid,
    similarity='similarity',
    input_fmt='cx_matrix',
    output_fmt='matrix',
    subset=None)

# Rank the similarity scores
sim_rank = pd.DataFrame(sim).rank(0) / (sim.shape[0] - 1)
sim_rank = pd.DataFrame((sim_rank.values + sim_rank.values.T) / 2.0, columns=sim_names, index=sim_names)

In [4]:
monarch = pd.read_table('/cellar/users/mikeyu/DeepTranslate/monarch/gene_disease.tsv', header=0)
monarch = monarch.loc[monarch['subject_taxon_label']=='Homo sapiens']
monarch = monarch.loc[[x.split(':')[0] not in ['NCBIGene', 'OMIA'] for x in monarch['object']]]
# Convert punctuation to underscores '_'
trantab = string.maketrans(string.punctuation, '_' * len(string.punctuation))
monarch_groups = list(monarch.groupby(monarch.apply(lambda x: x['object_label'].translate(trantab).lower().replace(' ', '_'), 1)))

19468 genes, 19343 terms, 215864 gene-term relations, 43296 term-term relations
node_attributes: Term_Description, Branch, Size, uniprot, ensembl_gene, entrez_gene
edge_attributes: 


# Make ontologies

In [27]:
for disease_index in range(len(monarch_groups)):
    
    disease_name, group = monarch_groups[disease_index]
    seed = group['subject_label'].unique().tolist()
    print 'Disease:', disease_name, '\n\t', 'Seed:', len(seed), seed

    total_genes = set(sim_names)
    seed = [s for s in seed if s in total_genes]
    print '\tFiltered seeds:', len(seed)

    expand, expand_idx, sim_2_seed, fig = ddot.expand_seed(seed, **{'sim': sim_rank.values, 'sim_names': sim_names, 'seed_perc':0, 'agg':'mean', 'figure':True})
    print '\tExpanded genes:', len(expand)

    if len(expand) <= 500:
        ont, url, ont_ndexgraph, expand_results = ddot.make_seed_ontology(
              seed,
              sim,
              sim_names,
              alpha,
              beta,
              go_human,
              node_attr=gene_drug,
              name=disease_name,
              expand_kwargs={'sim': sim_rank.values, 'sim_names': sim_names, 'seed_perc':0, 'agg':'mean', 'figure':True},
              ndex_kwargs={'ndex_user':ndex_user, 'ndex_pass':ndex_pass, 'ndex_server':ndex_server},
              align_kwargs={'iterations':100, 'threads': threads, 'update_self': ['Term_Description', 'Size']},
              subnet_max_term_size=None,
              verbose=True,
              ndex=False,
        )
        print url
        print ont

        basename = 'diseases/%s' % disease_name
        _ = ont.to_cx('%s.cx' % basename, layout='bubble-collect', style='passthrough')
        _ = ont.unfold().to_cx('%s.unfold.cx' % basename, layout='bubble', style='passthrough')
        pd.io.pickle.to_pickle(expand_results, '%s.expand.pkl' % basename)
        if ont_ndexgraph is not None:
            nx.write_graphml(ont_ndexgraph, '%s.graphml' % basename)
        with open('%s.ndex_url.txt' % basename, 'w') as f:
            f.write(url + '\n')
        ont.to_pickle('%s.pkl' % basename)

temp output: /tmp/tmpqYjD86
temp graph: /tmp/tmp0y49Yt
temp output log: /tmp/tmpHMj6I3
/tmp/tmp0y49Yt	0.05	0.5	-10000000 2017-11-09 17:02:23.972950
CLIXO command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/mhk7-clixo_0.3-cec3674/clixo /tmp/tmp0y49Yt 0.05 0.5 | awk '{if ( $1 ~ /^#/ ) {print "\#", strftime("%Y-%m-%d %H:%M:%S"), $0 ; fflush() } else {print $0}}' | tee /tmp/tmpHMj6I3
Extracting by grep -v # 2017-11-09 17:02:24.119528
Elapsed time (sec): 0.154404163361 2017-11-09 17:02:24.154821
Ontology: 3 genes, 1 terms, 3 gene-term relations, 0 term-term relations
node_attributes: 
edge_attributes: CLIXO_score
collapse command: /cellar/users/mikeyu/alignOntology/collapseRedundantNodes /tmp/tmp4hm_K6
collapse command: /cellar/users/mikeyu/alignOntology/collapseRedundantNodes /tmp/tmppPrKN4
Alignment command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/calculateFDRs /tmp/tmpPcFTTR /tmp/tmpnEw0EB 0.05 criss_cross /tmp/tmpRmkFSy 100 40 gene
Alignment: 1 matches
None
3 genes,

NameError: name 'ont_url' is not defined